In [ ]:
%pip install ultralytics

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import torch
import shutil
import json
import yaml

from PIL import Image
from sahi.utils.coco import Coco
from ultralytics import YOLO

torch.cuda.is_available()

True

In [2]:
DATA_DIR = os.path.join("..", "data")
IMAGE_DIR = "images_sliced" 
IMAGE_DIR_PATH = os.path.join(DATA_DIR, IMAGE_DIR)
YOLO_DIR_PATH = os.path.join(DATA_DIR, "yolo")

In [3]:
splits_dir = {
    "train": "train_sliced_coco.json",
    "val": "val_sliced_coco.json"
}

for split, anno_path in splits_dir.items():
    save_path = os.path.join(YOLO_DIR_PATH, split)

    if os.path.exists(save_path):
        shutil.rmtree(save_path)
    os.makedirs(os.path.join(save_path, "images"))
    os.makedirs(os.path.join(save_path, "labels"))

    with open(os.path.join(DATA_DIR, anno_path)) as f:
        train_coco = json.load(f)

    # get classes
    id2label = {label["id"]: label["name"] for label in train_coco["categories"]}
    label2id = {label["name"]: label["id"] for label in train_coco["categories"]}

    for img_meta in train_coco["images"]:
        img_path = os.path.join(IMAGE_DIR_PATH, img_meta["file_name"])

        # save image to yolo folder
        with Image.open(img_path) as img:
            width, height = img.size
            img.save(os.path.join(save_path, "images", img_meta["file_name"]))

        img_annos = [anno for anno in train_coco["annotations"] if anno["image_id"] == img_meta["id"]]

        for anno in img_annos:
            cls = anno["category_id"]

            # skip class: unknown
            # if cls == label2id["unknown"]:
            #     continue

            # convert x and y to yolo format and normalize bboxes
            x_min, y_min, w, h = anno["bbox"]
            x = (x_min + w / 2) / width
            y = (y_min + h / 2) / height
            w /= width
            h /= height

            with open(os.path.join(save_path, "labels", f"{img_meta['file_name'][:-4]}.txt"), "a") as f:
                f.write(f"{cls} {x} {y} {w} {h}\n")

In [4]:
yolo_config = {
    "path": os.path.join(os.getcwd(), YOLO_DIR_PATH),
    "train": "train/images",
    "val": "val/images",
    "test": "",
    "nc": len(label2id),
    "names": list(label2id.keys()),
}

with open(os.path.join(YOLO_DIR_PATH, "data.yaml"), "w") as f:
    yaml.dump(yolo_config, f, sort_keys=False)

In [ ]:
augmentations = {
    "hsv_h": 0.0,  # hue
    "hsv_s": 0.0,  # saturation
    "hsv_v": 0.0,  # value
    "degrees": 0.0,  # rotation
    "translate": 0.0,  # translate
    "scale": 0.0,  # scale
    "shear": 0.0,  # shear
    "perspective": 0.0,  # perspective
    "flipud": 0.0,  # flip up-down
    "fliplr": 0.0,  # flip left-right
    "mosaic": 0.0,  # mosaic
    "mixup": 0.0  # mixup
}

In [3]:
# Load a model
model = YOLO("../models/yolov10n.pt") 
results = model.train(
    data=os.path.join(YOLO_DIR_PATH, "data.yaml"), 
    imgsz=640, 
    device=0,
    epochs=100, 
    batch=16,
    optimizer="AdamW",
    lr0=0.01,
    patience=100,
    project="../results/yolo",
    exist_ok=True,
    plots=True,
    # **augmentations
)

Ultralytics 8.3.29  Python-3.10.7 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
engine\trainer: task=detect, mode=train, model=../models/yolov10n.pt, data=..\data\yolo\data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=../results/yolo, name=train, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

train: Scanning C:\Users\Admin\CodeProjects\GitHub\defect-inspection\data\yolo\train\labels.cache... 794 images, 518 backgrounds, 0 corrupt: 100%|██████████| 1312/1312 [00:00<?, ?it/s]
val: Scanning C:\Users\Admin\CodeProjects\GitHub\defect-inspection\data\yolo\val\labels.cache... 557 images, 531 backgrounds, 0 corrupt: 100%|██████████| 1088/1088 [00:00<?, ?it/s]


Plotting labels to ..\results\yolo\train\labels.jpg... 
optimizer: AdamW(lr=0.01, momentum=0.937) with parameter groups 95 weight(decay=0.0), 108 weight(decay=0.0005), 107 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to ..\results\yolo\train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      2.97G      6.047      10.57      4.679         53        640: 100%|██████████| 82/82 [00:14<00:00,  5.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:04<00:00,  7.35it/s]

                   all       1088       1506          0          0          0          0


...

                   all       1088       1506       0.17      0.141        0.1     0.0416

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      3.02G      3.618      4.719      2.925         31        640: 100%|██████████| 82/82 [00:13<00:00,  6.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:05<00:00,  6.60it/s]


                   all       1088       1506      0.173      0.134     0.0969     0.0423

100 epochs completed in 0.541 hours.
Optimizer stripped from ..\results\yolo\train\weights\last.pt, 5.8MB
Optimizer stripped from ..\results\yolo\train\weights\best.pt, 5.8MB

Validating ..\results\yolo\train\weights\best.pt...
Ultralytics 8.3.29  Python-3.10.7 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
YOLOv10n summary (fused): 285 layers, 2,698,706 parameters, 0 gradients, 8.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:04<00:00,  7.48it/s]


                   all       1088       1506      0.174      0.133     0.0968     0.0421
                bubble          5          5          0          0          0          0
            chip_crack         27         28      0.299      0.107     0.0854     0.0409
           light_stain         58         69          0          0    0.00581    0.00207
            line_crack        380        765      0.249       0.17      0.123     0.0449
     particle_material        288        454      0.404       0.41      0.378      0.172
         residue_stain         62         69     0.0843      0.159     0.0406    0.00939
               unknown        100        116      0.182     0.0862     0.0446     0.0255
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to ..\results\yolo\train


In [ ]:
# TODO: custom conversion

if os.path.exists(YOLO_DIR_PATH):
    shutil.rmtree(YOLO_DIR_PATH)

coco = Coco.from_coco_dict_or_path(ANNOTATION_PATH, image_dir=IMAGE_DIR_PATH)
coco.export_as_yolov5(output_dir=YOLO_DIR_PATH, train_split_rate=0.7)

indexing coco dataset annotations...


Loading coco annotations: 100%|██████████| 1312/1312 [00:00<00:00, 6593.08it/s]
11/12/2024 15:35:22 - INFO - sahi.utils.coco -   generating image symlinks and annotation files for yolov5...
100%|██████████| 918/918 [00:00<00:00, 976.71it/s] 
11/12/2024 15:35:23 - INFO - sahi.utils.coco -   generating image symlinks and annotation files for yolov5...
100%|██████████| 394/394 [00:00<00:00, 960.76it/s] 
